In [2]:
# ==== 1. Import thư viện cần thiết ====
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Nhập mô hình Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
import joblib
import os
import numpy as np

# Cấu hình tên file
DATA_FILE = "NaiveBayes_data.csv"
MODEL_FILE = "naive_bayes_pipeline.joblib"
TARGET_ENCODER_FILE = "nb_target_encoder.joblib"

In [ ]:
# ==== 2. Load dữ liệu ====
try:
    # Đọc dữ liệu. Giả sử cột ID không tồn tại hoặc đã được xử lý
    df = pd.read_csv(DATA_FILE)
    print(f"Đã load thành công {len(df)} dòng dữ liệu từ file {DATA_FILE}.")
    # print("-" * 50)
    # print(df.head()) 
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file {DATA_FILE}. Vui lòng kiểm tra lại tên file và đường dẫn.")

Đã load thành công 14 dòng dữ liệu từ file NaiveBayes_data.csv.
--------------------------------------------------
    Outlook Temperature Humidity    Wind Play ball
0     Sunny         Hot     High    Weak        No
1     Sunny         Hot     High  Strong        No
2  Overcast         Hot     High    Weak       Yes
3     Rainy        Mild     High    Weak       Yes
4     Rainy        Cool   Normal    Weak       Yes


In [4]:
# ==== 3. Phân tách Features (X) và Target (y) ====
target_col = "Play ball"
feature_cols = ["Outlook", "Temperature", "Humidity", "Wind"]

if target_col in df.columns and all(col in df.columns for col in feature_cols):
    X = df[feature_cols]
    y = df[target_col]
    
    print(f"\nFeatures: {list(X.columns)}")
    print(f"Target: {target_col}")

    # Xác định các cột phân loại (tất cả 4 cột trong ví dụ này)
    cat_cols = X.select_dtypes(include=['object']).columns

    # Mã hóa mục tiêu (Play ball: Yes/No) sang số nguyên
    target_encoder = LabelEncoder()
    y_encoded = target_encoder.fit_transform(y)
    print("Classes:", target_encoder.classes_)


Features: ['Outlook', 'Temperature', 'Humidity', 'Wind']
Target: Play ball
Classes: ['No' 'Yes']


In [5]:
# ==== 4. Định nghĩa Pipeline Tiền xử lý ====
# Sử dụng OneHotEncoder cho các biến phân loại
preprocessor = ColumnTransformer(
    transformers=[
        # sparse_output=False để đảm bảo đầu ra là mảng NumPy dày đặc (dense array), cần thiết cho GaussianNB
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols) 
    ],
    remainder='passthrough'
)

In [6]:
# ==== 5. Xây dựng Pipeline hoàn chỉnh (Sử dụng GaussianNB) ====
# Khởi tạo mô hình Gaussian Naive Bayes
model_nb = GaussianNB()

pipe_nb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model_nb)
])

In [7]:
# ==== 6. Chia tập train/test và Huấn luyện ====
# Chia dữ liệu, test_size=0.3 cho tập dữ liệu nhỏ
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42
)

print(f"\nKích thước tập Train: {len(X_train)} mẫu")
print(f"Kích thước tập Test: {len(X_test)} mẫu")

pipe_nb.fit(X_train, y_train)
print("Đã huấn luyện xong mô hình Gaussian Naive Bayes.")


Kích thước tập Train: 9 mẫu
Kích thước tập Test: 5 mẫu
Đã huấn luyện xong mô hình Gaussian Naive Bayes.


In [8]:
# ==== 7. Đánh giá mô hình ====
y_pred = pipe_nb.predict(X_test)
y_test_labels = target_encoder.inverse_transform(y_test)
y_pred_labels = target_encoder.inverse_transform(y_pred)

print("\n" + "="*15 + " KẾT QUẢ ĐÁNH GIÁ (NAIVE BAYES) " + "="*15)
print("Accuracy:", accuracy_score(y_test_labels, y_pred_labels))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred_labels, zero_division=0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_labels, y_pred_labels))
print("="*65)


=============== KẾT QUẢ ĐÁNH GIÁ (NAIVE BAYES) ===============
Accuracy: 0.8

Classification Report:
               precision    recall  f1-score   support

          No       1.00      0.50      0.67         2
         Yes       0.75      1.00      0.86         3

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5


Confusion Matrix:
 [[1 1]
 [0 3]]


In [9]:
# ==== 8. Lưu model Pipeline và Encoder ====
joblib.dump(pipe_nb, MODEL_FILE)
joblib.dump(target_encoder, TARGET_ENCODER_FILE)

print(f"\nPipeline model (Naive Bayes) đã được lưu thành công → {MODEL_FILE}")
print(f"Target Encoder đã được lưu thành công → {TARGET_ENCODER_FILE}")


Pipeline model (Naive Bayes) đã được lưu thành công → naive_bayes_pipeline.joblib
Target Encoder đã được lưu thành công → nb_target_encoder.joblib
